In [1]:
import numpy as np
import pandas as pd
import nltk
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [2]:
train_df = pd.read_csv(r"C:\Users\Hp\Documents\Python_scripts\NLP with disaster tweets/train.csv")
test_df = pd.read_csv(r"C:\Users\Hp\Documents\Python_scripts\NLP with disaster tweets/test.csv")

In [3]:
train_df.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
train_df[train_df['target'] == 0]['text'].values[1]

'I love fruits'

In [6]:
train_df[train_df['target'] == 1]['text'].values[500]

'Bush Fires are scary....even scarier when you go down and fight them'

In [7]:
nltk.download('punkt')
nltk.download('average_perceptron_tegger')
from nltk import word_tokenize, pos_tag

train_df_tokenize = [word_tokenize(word) for word in train_df['text']]
test_df_tokenize = [word_tokenize(word) for word in test_df['text']]

#train_df_tokens

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Error loading average_perceptron_tegger: Package
[nltk_data]     'average_perceptron_tegger' not found in index


In [8]:
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import re, string

def clean_tweets(tweets, stop_words = ()):
    
    lemmatizer = WordNetLemmatizer()
    clean_tweets = []
    for word,tag in pos_tag(tweets):
        
        word = re.sub('((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*','', word)
        word = re.sub("@(a-zA-Z0-9_)", '',word)
        word = re.sub('\w*\d\w*', '', word)
        word = re.sub('http|co|û_|amp', '', word)
        
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
            
        word = lemmatizer.lemmatize(word, pos)
        if len(word) > 0 and word.lower() not in stop_words and word not in string.punctuation:
            clean_tweets.append(word.lower())
            
    return clean_tweets
    

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

train_df_tokens = []
test_df_tokens = []

for token in train_df_tokenize:
    train_df_tokens.append(clean_tweets(token, stop_words))

for token in test_df_tokenize:
    test_df_tokens.append(clean_tweets(token, stop_words))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def list_to_string(token):
    space = " "
    s = []
    for text in token:
        s.append(space.join(text))
    return s

train_tokens_str = list_to_string(train_df_tokens)
test_tokens_str = list_to_string(test_df_tokens)

In [11]:
train_df['clean_text'] = train_tokens_str
test_df['clean_text'] = test_tokens_str
train_df.head()


,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason earthquake may allah forgive us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident ask 'shelter place notify officer eva...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,get send photo ruby alaska smoke wildfire pour...


In [12]:
tf_idf = feature_extraction.text.TfidfVectorizer()

example_train_vectors = tf_idf.fit_transform(train_df['clean_text'][0:5])

print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 35)
[[0.         0.37796447 0.         0.         0.         0.37796447
  0.37796447 0.         0.         0.         0.         0.37796447
  0.         0.         0.37796447 0.         0.         0.
  0.         0.         0.         0.         0.         0.37796447
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.37796447 0.        ]]


In [13]:
train_vectors = tf_idf.fit_transform(train_df["clean_text"])
test_vectors = tf_idf.transform(test_df["clean_text"])

In [14]:
clf = linear_model.RidgeClassifier()

scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=5, scoring="f1")
scores

array([0.59168242, 0.54637437, 0.58766234, 0.52812202, 0.69310071])

In [15]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [16]:
sample_submission_1 = pd.read_csv(r"C:\Users\Hp\Documents\Python_scripts\NLP with disaster tweets/sample_submission.csv")

In [17]:
sample_submission_1["target"] = clf.predict(test_vectors)

In [18]:
sample_submission_1.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [19]:
sample_submission_1.to_csv("submission.csv", index=False)